In [1]:
import torch
import torch.nn as nn
import numpy as np
import os

import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import torch.utils.data


In [13]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    losses = []
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        loss = model.loss(output, label)
        losses.append(loss.item())
        loss.backward()
        
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(losses)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += model.loss(output, label, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(label.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / (len(test_loader.dataset) * test_loader.dataset.sequence_length)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, test_accuracy


In [3]:
LR = 0.9
WEIGHT_DECAY = 0.0001
MOMENTUM = 0.9
EPOCHS = 5000
BATCH_SIZE = 16
USE_CUDA = True
SEED = 0
LOG_INTERVAL = 100


In [4]:
use_cuda = USE_CUDA and torch.cuda.is_available()
torch.manual_seed(SEED)
device = torch.device("cuda" if use_cuda else "cpu")
print('Using device', device)

import multiprocessing
print('num cpus:', multiprocessing.cpu_count())
kwargs = {'num_workers': multiprocessing.cpu_count(),
          'pin_memory': True} if use_cuda else {}


Using device cuda
num cpus: 8


In [14]:
model_conv = torchvision.models.resnet18(pretrained=True)

# avoid training pretrained model
for param in model_conv.parameters():
    param.requires_grad = False

# set categories to 2
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_conv.fc.parameters(), 
                      lr=LR, 
                      momentum=MOMENTUM, 
                      weight_decay=WEIGHT_DECAY)

model_conv.loss = criterion


In [9]:
INPUT_SIZE = (224, 224)


class ImageLoader(object):

    def __init__(self, batchSize):
        super(ImageLoader, self).__init__()
        transform_train = transforms.Compose([
            transforms.Resize(size=INPUT_SIZE),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        transform_test = transforms.Compose([
            transforms.Resize(size=INPUT_SIZE),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        train_dataset = datasets.ImageFolder(root='chest_xray_dataset/train',
                                             transform=transform_train)
        self.trainloader = torch.utils.data.DataLoader(train_dataset,
                                                       batch_size=batchSize,
                                                       shuffle=True,
                                                       num_workers=2)

        test_dataset = datasets.ImageFolder(root='chest_xray_dataset/test',
                                            transform=transform_test)
        self.testloader = torch.utils.data.DataLoader(test_dataset,
                                                      batch_size=batchSize,
                                                      shuffle=False,
                                                      num_workers=2)

        self.classes = ('normal', 'pneumonia')


In [12]:
loader = ImageLoader(batchSize=BATCH_SIZE)

try:
    for epoch in range(EPOCHS + 1):
        train(model_conv, device, loader.trainloader, optimizer, epoch, LOG_INTERVAL)
        test(model_conv, device, loader.testloader)
except KeyboardInterrupt as ke:
    print('Interrupted')
except:
    import traceback
    traceback.print_exc()
finally:
    pass


Train Epoch: 0 [0/5216 (0%)]	Loss: 0.776940


Train Epoch: 0 [1600/5216 (31%)]	Loss: 53.463085


Train Epoch: 0 [3200/5216 (61%)]	Loss: 4.222226


Train Epoch: 0 [4800/5216 (92%)]	Loss: 13.006910


Traceback (most recent call last):
  File "<ipython-input-12-1f7be6a7b070>", line 6, in <module>
    test(model_conv, device, loader.testloader)
  File "<ipython-input-11-d95dfd37421f>", line 30, in test
    test_loss += model.loss(output, label, reduction='sum').item()
  File "/home/liangyu/.local/lib/python3.6/site-packages/torch/nn/modules/module.py", line 532, in __getattr__
    type(self).__name__, name))
AttributeError: 'ResNet' object has no attribute 'loss'
